# FX Market Making Analytics

**Author:** Mame Faty Lo | 1290, Versoix | mamefatylo.b@gmail.com  

This notebook explores **FX market making, liquidity, and trading analytics** using Python.  

---

## Objectives

- Analyze **FX market data** for Swissquote-relevant pairs (EUR/USD, GBP/USD, USD/CHF, etc.).
- Explore **bid-ask spreads, mid-prices, volatility, and order book behavior**.
- Simulate **market making strategies** and liquidity provision.
- Generate **interactive charts and static figures** to support insights

---
>**Note**
>- All analyses are **fully reproducible** using Conda (`fx_env.yml`).
>- Data is saved in `data/raw/` (original) and `data/processed/` (cleaned).
>- Summaries are saved in `reports/figures/` and `reports/summaries/`.

## 1. Setup

In [8]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import yfinance as yf         # fetch financial data
import plotly.io as pio

# Display options
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", "{:.4f}".format)

# Seaborn & Matplotlib defaults
sns.set_context("notebook")
sns.set_style("darkgrid")

# Plotly defaults
pio.templates.default = "plotly_dark"
px.defaults.width = 900
px.defaults.height = 500
px.defaults.color_continuous_scale = px.colors.sequential.Viridis

In [ ]:
# Define project folders (portable across machines)
PROJECT_ROOT = Path.cwd()
DATA_RAW = PROJECT_ROOT / "data/raw"
DATA_PROCESSED = PROJECT_ROOT / "data/processed"
REPORTS_SUM = PROJECT_ROOT / "reports/summaries"
REPORTS_FIG = PROJECT_ROOT / "reports/figures"

# Create folders if they don't exist
for folder in [DATA_RAW, DATA_PROCESSED, REPORTS_SUM, REPORTS_FIG]:
    folder.mkdir(parents=True, exist_ok=True)

In [7]:
# Define FX pair and period
PAIR = "USDCHF=X"
START_DATE = "2025-01-01"
END_DATE = "2025-10-24"

# Download daily historical data
data = yf.download(
    PAIR,
    start=START_DATE,
    end=END_DATE,
    interval='1d',
    auto_adjust=False   # keep raw prices
)

# Keep relevant columns
data = data[['Open','High','Low','Close','Volume']]

# Preview
data.head()

[*********************100%***********************]  1 of 1 completed


Price,Open,High,Low,Close,Volume
Ticker,USDCHF=X,USDCHF=X,USDCHF=X,USDCHF=X,USDCHF=X
Date,,,,,
2025-01-02,0.90775,0.91365,0.90395,0.90775,0
2025-01-03,0.91201,0.91243,0.90810,0.91201,0
2025-01-06,0.90931,0.91044,0.90100,0.90931,0
2025-01-07,0.90504,0.90955,0.90227,0.90504,0
2025-01-08,0.90945,0.91280,0.90870,0.90945,0


## Data Cleaning

We first check for missing values in the USD/CHF data. Any missing prices are filled using forward-fill to maintain continuity. This ensures calculations like returns and volatility are accurate.

In [15]:
data.isnull().sum()

Price   Ticker  
Open    USDCHF=X      0
High    USDCHF=X      0
Low     USDCHF=X      0
Close   USDCHF=X      0
Volume  USDCHF=X      0
Return              210
dtype: int64

## Basic Statistics

Summary statistics give an overview of USD/CHF price behavior. 
We can see average prices, variability, and ranges, which are useful for understanding market dynamics.

In [16]:
data.describe()

Price,Open,High,Low,Close,Volume,Return
Ticker,USDCHF=X,USDCHF=X,USDCHF=X,USDCHF=X,USDCHF=X,
count,210.000000,210.000000,210.000000,210.000000,210.0,0.0
mean,0.837796,0.840446,0.834484,0.837796,0.0,NaN
std,0.043675,0.043604,0.043406,0.043675,0.0,NaN
min,0.785910,0.787800,0.785700,0.785910,0.0,NaN
25%,0.801440,0.803417,0.798517,0.801440,0.0,NaN
50%,0.820590,0.823305,0.816555,0.820590,0.0,NaN
75%,0.882863,0.884700,0.880067,0.882863,0.0,NaN
max,0.916770,0.919940,0.915400,0.916770,0.0,NaN


## Daily Returns

Daily returns measure the percentage change in closing price from one day to the next. 
They are crucial for analyzing market movement and calculating risk metrics.

In [17]:
data['Return'] = data['Close'].pct_change()

# Preview returns
data[['Close','Return']].head()

Price,Close,Return
Ticker,USDCHF=X,
Date,,
2025-01-02,0.90775,NaN
2025-01-03,0.91201,NaN
2025-01-06,0.90931,NaN
2025-01-07,0.90504,NaN
2025-01-08,0.90945,NaN
